## PORTFÖY TEORİSİ DENEME II
Elektrik fiyat değerleri EPİAŞ'ın şeffaflık platformundan API formatında elde edildi.  Bu çalışmanın temel amacı uygun bir portföy oluşturmaktır. Hem üretim hemde tüketim bacağı için optimum portföyün oluşması hedeflenmektedir. Burada kullanılan yöntem ilgili bölümde açıklanacaktır.

# 1.1 Verilerin Elde Edilmesi
Bu çalışma farklı üretim tiplerinde uygulamada olabilecek fiyatlar ile (uzman görüşü ile makul seviyede olacak şekilde) hedef bir portföyün oluşturmasını hedeflemektedir. Portföy dizayn teoriye göre formüller elde edilmiş ve bu formüller denenmiştir. 
+ İlk olarakkütüphaneler yüklendi.
+ request için pip install çalıştı.
+ Verinin elde edilmesi için bir çok metot denendiğinden bazı kütüphaneler gereksiz kalmış olabilir.

+ Belirtilen tarihe göre PTF değerleri TRY, USD ve EUR olarak elde edildi.
+ Veri saatliktir ve ortalama değerleri kullanılacaktır.
+ Verilern JSON formatında elde edildikten sonra dataframe olarak depolanmıştır.
+ İlk demeneme olduğu için tarih kısa tutuldu. 

In [364]:
import pandas as pd
import requests as rq
import json
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
from matplotlib import pyplot
import statsmodels.api as sm
from scipy.fft import fft, fftfreq
from retrying import retry
import requests 

In [365]:
url = "https://giris.epias.com.tr"
 
headertgt = {"Content-Type" : "application/x-www-form-urlencoded",
             "Accept" : "text/plain"}
 
paramtgt = {"username" : "ademyildirimmakine@gmail.com",
            "password" : "Ay.12345"}
 
TGT_response = requests.post(url+"/cas/v1/tickets", headers = headertgt, params = paramtgt)

In [366]:
date1 = "2025-02-01T00:00:00+03:00"
date2 = "2025-02-28T00:00:00+03:00" #sontarihin tamamını alıyor
base_path = "https://seffaflik.epias.com.tr/electricity-service"
header={"Content-Type" : "application/json",
        "Accept" : "application/json",
        "Accept-Language" :"en",
        "TGT" : TGT_response.text}

Toplam ihtiyaç C bu örnekte 120 olarak seçildi. 

4 Farklı fiyatta ürün seçildi.
+ Doğrudan PTF
+ Sabit EUR => 67 EUR/MWh
+ Sabit USD => 70 EUR/MWh
+ Sabit TRY => 2400-2600 TL/MWh

In [367]:
ptfparam1  = json.dumps({"startDate": date1, "endDate": date2,"page": {"number": "", "size": "", "total": "", "sort": {"field": "date",  "direction": "ASC"}}})
ptf1  = rq.request("POST", base_path+"/v1/markets/dam/data/mcp",  data = ptfparam1, headers=header)
ptf1  = ptf1.json()
ptf   = pd.DataFrame(ptf1["items"])

In [368]:
ptf["USDTRY"] = ptf["price"]/ptf["priceUsd"]
ptf["EURTRY"] = ptf["price"]/ptf["priceEur"]
ptf["S1"] = ptf["EURTRY"] * 80
ptf["S2"] = ptf["USDTRY"] * 81
ptf["S3"] = ptf["price"]*1.2
ptf["hour"] = ptf["hour"].str[:2].astype('float')
ptf['S4'] = ptf['hour'].apply(lambda x: 3500  if x >=12  else 2500 )
#ptf.describe()

In [369]:
ptf["S4"].var()
satıcılar = ptf[["S1","S2","S3","S4"]]
P   = pd.DataFrame([ptf["S1"].mean(),ptf["S2"].mean(),ptf["S3"].mean(),ptf["S4"].mean()], columns=['Ave Price'])
BV  = pd.DataFrame([1,1,1,1], columns=['Birim Vektör'])
cov = satıcılar.cov()
C   = 5000
a   = 100
satıcılar.describe()
#satıcılar.plot()

,S1,S2,S3,S4
count,672.000000,672.000000,672.000000,672.00000
mean,3003.590376,2920.044927,2973.935643,3000.00000
std,32.453059,16.611153,499.653127,500.37244
min,2940.821831,2893.239847,900.000000,2500.00000
25%,2972.335709,2908.200756,2639.988000,2500.00000
50%,2996.916191,2918.930196,3000.012000,3000.00000
75%,3030.133111,2934.752560,3418.347000,3500.00000
max,3054.303455,2947.613159,3600.000000,3500.00000


In [370]:
T_P    = P.values.T
T_BV   = BV.values.T
in_cov = np.linalg.inv(cov.values)
#print("P Transpose" , T_P.shape)
#print("T_BV Transpose" , T_BV.shape)


In [371]:
u = ((np.matmul(np.matmul(T_BV, in_cov),P.values)*a/(2*C)) + 1 ) / (np.matmul(np.matmul(T_BV, in_cov),BV.values))
k = ((u*BV.values) - ( P.values*(1/(2*C))))
λ = np.matmul(in_cov, k)
λ

array([[-7.92342839e-01],
       [ 2.00473197e+00],
       [ 1.45067162e-03],
       [ 5.13254308e-04]])

In [372]:
E1 = np.matmul(λ.T, P)*a*C
E2 = C * ptf["S3"].mean()
Var1 =(C^2) * (np.matmul(np.matmul(λ.T, in_cov),λ))
print("Beklenen Birim Değer" , E1.values/C/a)
print("Beklenen Birim Değer" , E2/C)

Beklenen Birim Değer [[3479.88805143]]
Beklenen Birim Değer 2973.935642857143


## 1.2 Portföy Tasarımı
Normal portföy teorisinde (hisse senedi yatırımı gibi) amaç:
+ Beklenen kazancı maksimum yapmak ve
+ Riski (varyansı) minimum yapmak.

Ancak burada amaç biraz farklı:
+ Hem ortalama fiyatı (beklenen maliyet) düşük olsun birde
+ Hem de fiyat oynaklığı (risk) düşük olsun.

Bunu sağlamak için minimize edilen amaç fonksiyonu aşağıdaki gibidir
amaç fonksiyonu => a⋅E(X/λ)+Var(X/λ)
Burada:
+ E(X): Expected Value ya da ortalama maliyettir.
+ Var(X): X'in varyansı başka bir deyişle X'in riskidir.
+ a: senin ortalama fiyata verdiğin önem (ne kadar “ucuzluk” peşindesin).

Burada amaç fonksiyonu detaylandırılırsa ki aşağıdaki formülde 2 farklı satıcı için bu fonksşyon yazılmıştır. Satıcı S1 ve p1 fiyatında, satıcı S2 ve p2 fiyatında. Toplam maliyeti hem ortalama fiyat hem risk açısından minimize etmek istiyoruz. Fonksiyon aşağıdaki gibidir.

+ F(λ)= aCλp1 + aC(1−λ)p2 + (C^2)(λ^2)Var(p1)+(C^2)((1−λ)^2)Var(p2)+2(C^2)λ(1−λ)Cov(p1,p2)

Burada:
+ İlk iki terim → beklenen maliyet (a⋅E(X))
+ Diğer üç terim → varyans (risk) 'dır.
Buradaki F fonksiyonunun λ'ya göre 1. türevini alıp 0'a eşitlediğimizde optimum λ'yı bulmuş olacağız. 

Türevivi aldığımızda aşağıdaki eşitliği elde ediyoruz.

+ dF/dλ =a(p1−p2)+2CλVar(p1)−2C(1−λ)Var(p2)+2C(1−2λ)Cov(p1,p2)

Birinci türev 0'a eşitlendiğinde ise optimum nokta bulunacak. 
Bu denklemi λ’ya göre çözdüğümüzde aşağıdaki eşitlik elde edilmektedir. 

+ λ= (a(p1−p2)/[2C[Var(p1)+Var(p2)−2Cov(p1,p2)]])+((Var(p2)−Cov(p1,p2))/[Var(p1)+Var(p2)−2Cov(p1,p2)])

Bir kısıtımız var ve bu kısıt λi = 1'dir. Bu yüzden Lagrange çarpanları ile çözülüyor. Sonuç olarak aşağıdaki denkleme evriliyor eşitlik.

+ ∑ λl*Cov(pl,pj)+apj/2C = λ

Yani her satıcı için bir denklem çıkıyor.

Lagrange çarpanları: Çok değişkenli fonksiyonların minimum ve maksimum noktalarını bulmaya yarar. 

+ Cov⋅L=λ1−2CaP

- Cov = kovaryans matrisi (fiyatların birlikte nasıl değiştiğini gösteren kare matris).
- L = (𝜆1,𝜆2,…,𝜆𝑛)T  (oranlar sütun vektörü).
- P = (𝑝1,𝑝2,…,𝑝𝑛)T (fiyatların ortalama değerleri).
- 1 = (1,1,1,…,1)^T (birlerden oluşan vektör).

Kovaryans matrisi (cov değişkeni ile adlandırılan) elde edildi. Price değerleri P vektörü ile gösteridi. λ vektörüde λ olarak tanımlandı. Lagrange opratörü olan λ ise u olarak temsil edildi. Bu kısım diğer diğer λ ile karıştırılmamalıdır. λ vektörü satıcıların ağırlıklarını vermektedir. 

f => a.C.λ.(P^T) + 2.(C^2).(λ^T).cov.λ
g => (1^T).λ = 1

Bu eşitlikte g fonsksiyonu sıfıra eşitlenip aşağıdaki lagrange eşitliği yazılırsa sonuç aşağıdaki gibi olur. Lagrange fonksiyonu L ile gösterildi. L fonsiyonunun türevi alındığında ve sıfıra eşitlendiğinde max ya da minimum noktalar bulunacaktır. Cov matrisinin tersi cov- olarak gösterildi. 

1. L = a.C.λ.(P^T) + 2.(C^2).(λ^T).cov.λ - u.((1^T).λ-1) (Genel matrisli denklem)

2. dL/dλ = a.C.(P^T) + 2.(C^2).cov.λ - 1.u  (Lagrange fonksiyonunun türevi alındı)

3. λ     = [ [ 1.u - a.C.(P^T) ] / 2.(C^2) ] . cov-  (dL 0'a eşitlenip λ eşitliği elde edildi.)

4. λ     = cov- . ((λ* .1) - a.P/2C)   (u skaler bir değer olduğu için u/2.(C^2) için λ* değişkeni kullanıldı.

5. (1^T).λ = 1 => (1^T) . cov- . ((λ* .1) - a.P/2C) (kısıt denkleminde λ yerine konuldu)

6. λ*    = [ 1(scaler) + (1^T).(cov-).(P).a/2C ] / [ (1^T).(cov-).(1) ]





